In [1]:
from single_bet_type_analyzer import SingleBetTypeAnalyzer
from dask.distributed import LocalCluster
from scrapers.betflag import BetflagScraper
from scrapers.betfair import BetfairScraper
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import winsound
import time

def play_beep():
    for i in range(3):
        for j in range(2):
            winsound.Beep(1000, 100)
            time.sleep(0.05)
        time.sleep(0.7)
        
def clean_df(_df, wealth):
    _df = _df.drop(columns=['sport', 'back_size', 'lay_size', 'prob',
                            'p2', 'p1', 'r1', 'r2']).rename(columns={'bet_type': 'scommessa',
                                                                     'back_price': 'punta',
                                                                     'site1': 'sito_punta',
                                                                     'lay_price': 'banca',
                                                                     'site2': 'sito_banca',
                                                                     'q1': 'puntata',
                                                                     'q2': 'bancata'})
    _df['puntata'] *= wealth
    _df['bancata'] *= wealth
    return _df

wealth = 10

cluster = LocalCluster(processes=False)
cluster

C:\Users\miche\anaconda3\envs\webautomation\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63192 instead
  warnings.warn(


In [2]:
sport = widgets.Dropdown(
    options=['calcio', 'tennis', 'basket'],
    value='calcio',
    description='Sport:',
    disabled=False,
)
bet_type = widgets.Dropdown(
    options=['1x2', '12', 'uo1.5', 'uo2.5', 'uo3.5', 'uo4.5'],
    value='1x2',
    description='Bet Type:',
    disabled=False,
)
offline = widgets.Checkbox(
    value=True,
    description='Non-Live',
    disabled=False
)

live = widgets.Checkbox(
    value=True,
    description='Live',
    disabled=False
)

headless = widgets.Checkbox(
    value=True,
    description='Headless',
    disabled=False
)
widgets.VBox([sport, bet_type, offline, live, headless])

In [5]:
%%time
try:
    if (sport.value != 'calcio' and bet_type.value != '12') or (sport.value == 'calcio' and bet_type.value == '12'):
        print('Errore: coppia sport-bet_type non accettata!')
        analyzer = None
    else:
        analyzer = SingleBetTypeAnalyzer(sport=sport.value, bet_type=bet_type.value, cluster=cluster, 
                                         offline=offline.value, live=live.value, headless=headless.value)
    # analyzer = SingleBetTypeAnalyzer('basket', '12', cluster=cluster)
except:
    print("Errore: Analyzer NON Creato, Riprovare!")
    # analyzer.close()
finally:
    print('Fine')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [C:\Users\miche\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [C:\Users\miche\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [C:\Users\miche\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache




Errore: Analyzer NON Creato, Riprovare!
Fine
Wall time: 1min 31s


In [ ]:
%%time
df = analyzer.analyze_bets()
clean_df(df, wealth)

In [4]:
while True:
    df = analyzer.analyze_bets()
    clear_output()
    display(clean_df(df, wealth))
    if len(df) > 0:
        play_beep()
        time.sleep(50)
    time.sleep(10)

,club1,club2,date,scommessa,sito_punta,punta,sito_banca,banca,prob,ROI,puntata,bancata
0,flamengo,paulo sao,2021-07-25,1,betflag,8.4,betfair,1.18,0.27159,1.4534,3.0,1.298969


KeyboardInterrupt: 

In [6]:
analyzer.close()

In [ ]:
cluster.close()

In [8]:
play_beep()